In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
for i in range(len(data)):
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'

    page = requests.get( url, headers=headers )

    # Beautiful Object
    soup = BeautifulSoup( page.text, 'html.parser' )

    # =============================== Color Name ===============================
    list_item = soup.find_all( 'a', role="radio" )

    color_name = [item.get( 'data-color' ) for item in list_item]
    product_id = [item.get( 'data-articlecode' ) for item in list_item]

    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']

    # Generate Style ID + Color ID
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

    # =============================== Composition ===============================


    product_composition_list = soup.find_all( 'div', class_='details-attributes-list-item' )
    product_composition = [list(filter(None, item.get_text().split( '\n' ) ) ) for item in product_composition_list]

    #Creating Dataframe
    
    if product_composition != []:
        df_composition1 = pd.DataFrame( product_composition ).T
        df_composition1.columns = df_composition1.iloc[0, :]
        df_composition = df_composition1[['Fit', 'Composition', 'Art. No.']]
        df_composition = df_composition[df_composition['Composition'].notnull()]

        #Deleting first row and pasting Fit and Art. No.
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )

        # Generate Style ID + Color ID
        df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3] )
        df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:] )

        # Merge Color + Composition
        data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id' )

        # All details products
        df_final = pd.concat( [df_final, data_sku], axis=0 )
    else:
        None

NameError: name 'data' is not defined